In [1]:
import cv2
from deepface import DeepFace
from insightface.app import FaceAnalysis


/opt/anaconda3/envs/deepface_env/lib/python3.9/site-packages/mtcnn/mtcnn.py:34: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:

face_app = FaceAnalysis(name="buffalo_l")
face_app.prepare(ctx_id=0)


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/naniathoti/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/naniathoti/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/naniathoti/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/naniathoti/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/naniathoti/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3,

In [3]:

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise RuntimeError("❌ Camera not accessible")

print("Press 'q' to quit")


Press 'q' to quit


In [4]:

while True:
    ret, frame = cap.read()
    if not ret:
        break

    
    faces = face_app.get(frame)

    for face in faces:
        x1, y1, x2, y2 = map(int, face.bbox)

       
        face_img = frame[y1:y2, x1:x2]
        if face_img.size == 0:
            continue

        
        age = int(face.age)
        gender = "Male" if face.gender == 1 else "Female"

        
        try:
            emotion_result = DeepFace.analyze(
                face_img,
                actions=["emotion"],
                enforce_detection=False,
                silent=True
            )

            if isinstance(emotion_result, list):
                emotion_result = emotion_result[0]

            emotion = emotion_result["dominant_emotion"]

        except Exception:
            emotion = "Unknown"

        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        label = f"Age: {age} | Gender: {gender} | Emotion: {emotion}"

        cv2.putText(
            frame,
            label,
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 255, 0),
            2
        )

    cv2.imshow("Live Face Analysis", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
